In [ ]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 7"
#import os
#os.environ["OMP_NUM_THREADS"] = "1"
#os.environ["MKL_NUM_THREADS"] = "1"
#import sklearn
import torch
import re
import wandb


from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
from datasets import load_dataset
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
# from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
# from transformers import LlamaForCausalLM, LlamaTokenizer

# from accelerate import Accelerator

# from Prompt import Prompt

# import torch
# import bitsandbytes as bnb
# import fire
# import random

In [15]:
import os
import random
random.seed(1958)
class Prompt:
    def __init__(self, prompt_path) -> None:
        assert os.path.isfile(prompt_path), "Please specify a prompt template"
        with open(prompt_path, 'r') as f:
            raw_prompts = f.read().splitlines()
        self.templates = [p.strip() for p in raw_prompts]
            
        self.historyList = []
        self.itemList = []
        self.trueSelection = ""

    def __str__(self) -> str:
        prompt = self.templates[random.randint(0, len(self.templates)-1)]
        history = ", ".join(self.historyList)
        cans = ", ".join(self.itemList)
        prompt = prompt.replace("[HistoryHere]", history)
        prompt = prompt.replace("[CansHere]", cans)
        prompt += " "
        return prompt  
def convert_dict_to_prompt(d:dict):
        prompt_path="S-DPO/prompt/movie.txt"  
        t = Prompt(prompt_path)
        d["historyList"] = d["historyList"].split("::") if isinstance(d["historyList"], str) else d["historyList"]
        t.historyList = d["historyList"]
        t.itemList = d["itemList"]
        t.trueSelection = d["trueSelection"]
        return t

def process_data(data_point):
    t = convert_dict_to_prompt(data_point)
    prompt = str(t)
    target = data_point["trueSelection"]
    dic = {
        "prompt": prompt,
        "completion": target
    }
    return dic

In [4]:
data_files = {
    "train": "./data/lastfm-sft-cans20/lastfm-train.json",
    "validation": "./data/lastfm-sft-cans20/lastfm-val.json",
}

data = load_dataset("json", data_files=data_files)

In [5]:
data['train'][0]

{'itemList': ['Dolphin',
  'HEARTSREVOLUTION',
  'Simple Plan',
  'M. Pokora',
  'Shout Out Louds',
  'Orient Expressions',
  'Peter Bjorn and John',
  'Mägo de Oz',
  'The Boo Radleys',
  'Baroness',
  '7Seconds',
  "Israel Kamakawiwo'ole",
  'Fleetwood Mac',
  'Julie London',
  'Big Mama Thornton',
  'David Lee Roth',
  'Burning Spear',
  'Dolly Parton',
  'Terri Clark',
  'Get Cape. Wear Cape. Fly'],
 'historyList': 'Morcheeba::Enigma::Café Del Mar',
 'trueSelection': 'Fleetwood Mac'}

In [16]:
train_data = data["train"].shuffle(seed=42).map(process_data)
train_data = train_data.remove_columns(data["train"].column_names)
print(train_data)

Map:   0%|          | 0/47872 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 47872
})


In [18]:
print(train_data[0])

{'prompt': 'After watching Katy Perry, P!nk, Allison Iraheta, The Pussycat Dolls, Backstreet Boys, Savage Garden, Simple Plan, Adam Lambert, Jewel, John Williams, the person is presented with the following movie options We Are The Ocean, Bill Evans, The Tragically Hip, KC and the Sunshine Band, Alan Silvestri, The Alarm, Lacuna Coil, Azam Ali, Autograph, Slagsmålsklubben, Nelly, Bif Naked, Психея, Samantha Fox, Hooverphonic, The Fall, Beseech, The Ditty Bops, Mariana Aydar, Air. Can you predict her/his next choice?  Answer: ', 'completion': 'Alan Silvestri'}
